In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
from torch import nn
from torch.nn.utils import prune
import numpy as np
import sys
import matplotlib.pyplot as plt
import math
import random
import sklearn.metrics as perf
import os
import cv2
import time
import torch_pruning as tp
import csv

from models.models import MTLClassifier, AgeRegressor, GenderClassifier, EthnicityClassifier
from utils.data import FacesDataset, data_transform
from utils.training import train_mtl_model, train_age_model, train_gender_model, train_ethnicity_model
from utils.evaluation import run_evaluation, show_example_predictions
from utils.pruning import prune_model, prune_other_tasks, get_f1_and_lat

### Load and Prepare Data

In [2]:
### Load in the data
folder = 'UTKFace'
transform = data_transform()
dataset = FacesDataset(folder=folder, transform=transform)

In [3]:
### Set up train and val datasets and loaders
train_len = int(len(dataset)*0.8)
val_len = len(dataset) - train_len
train_dataset, val_dataset = random_split(dataset, [train_len, val_len], torch.Generator().manual_seed(8))

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)

In [4]:
### Get small subset fpr train_loader and val_loader (for testing and debugging)
# train_indices = torch.randperm(len(train_dataset))[:100]
# val_indices = torch.randperm(len(val_dataset))[:100]
# train_subset = torch.utils.data.Subset(train_dataset, train_indices)
# val_subset = torch.utils.data.Subset(val_dataset, val_indices)
# train_loader = DataLoader(dataset=train_subset, batch_size=16, shuffle=True)
# val_loader = DataLoader(dataset=val_subset, batch_size=16, shuffle=True)

### MTL Model Variants

#### Define pruned training function

In [5]:
def pruned_mtl_training(task, prune_pct, num_epochs,
                        train_loader=train_loader, val_loader=val_loader,
                        val_dataset=None):
    
    ### Set up model, loss, and optimizer
    if task.upper()=='GENDER':
        tasks = ['gender']
        model = GenderClassifier()
        model = model.cuda()
        gender_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_SingleTaskModel.pth'
        train_gender_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                           train_loader=train_loader, val_loader=val_loader,
                           gender_criterion=gender_criterion, gender_coeff=1.0,
                           save=True, save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='age', task2='ethnicity', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_SingleTaskModel.pth'
        train_gender_model(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                           train_loader=train_loader, val_loader=val_loader,
                           gender_criterion=gender_criterion, gender_coeff=1.0,
                           save=True, save_name=save_initial)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                   eval_dataset=val_dataset,
                                                   eval_dataloader=val_loader,
                                                   tasks=tasks,
                                                   mtl_model=False)
        
        # Save model with score and latency information in the model name
        genderf1 = scores['gender'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, 0.0, genderf1, 0.0]
        with open('models/model_variants/model_score_lookup_singletask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_SingleTaskModel.pth'
        torch.save(model, f"models/{new_name}")
        
    elif task.upper()=='ETHNICITY':
        tasks = ['ethnicity']
        model = EthnicityClassifier()
        model = model.cuda()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_SingleTaskModel.pth'
        train_ethnicity_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              ethnicity_criterion=ethnicity_criterion, ethnicity_coeff=1.0,
                              save=True, save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='age', task2='gender', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_SingleTaskModel.pth'
        train_ethnicity_model(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              ethnicity_criterion=ethnicity_criterion, ethnicity_coeff=1.0,
                              save=True, save_name=save_initial)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                       eval_dataset=val_dataset,
                                                       eval_dataloader=val_loader,
                                                       tasks=tasks,
                                                       mtl_model=False)
        
        # Save model with score and latency information in the model name
        ethnicityf1 = scores['ethnicity'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, 0.0, 0.0, ethnicityf1]
        with open('models/model_variants/model_score_lookup_singletask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_SingleTaskModel.pth'
        torch.save(model, f"models/{new_name}")
        
    elif task.upper()=='AGE':
        tasks = ['age']
        model = AgeRegressor()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_SingleTaskModel.pth'
        train_age_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              age_criterion=age_criterion, age_coeff=1.0,
                              save=True, save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='gender', task2='ethnicity', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_SingleTaskModel.pth'
        train_age_model(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              age_criterion=age_criterion, age_coeff=1.0,
                              save=True, save_name=save_initial)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                       eval_dataset=val_dataset,
                                                       eval_dataloader=val_loader,
                                                       tasks=tasks,
                                                       mtl_model=False)
        
        # Save model with score and latency information in the model name
        ager2 = scores['age'][1]
        row = [task.upper(), prune_pct, mean_lat, std_lat, ager2, 0.0, 0.0]
        with open('models/model_variants/model_score_lookup_singletask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_SingleTaskModel.pth'
        torch.save(model, f"models/{new_name}")
        
    else:
        print('Invalid task was specified.')
        
    # Delete intermediate model files
    os.remove(f'models/{save_initial}')
    if save_initial != save_no_prune:
        os.remove(f'models/{save_no_prune}')
        
    return scores, mean_lat, std_lat
        


#### 0% to 90% Pruning

Notes: pruning the linear task-specific layers was sometimes causing an index-out-of-bounds error, which I think is due to it pruning off the final layer. As a fix, trying ignoring the last layer when pruning the linear layers. Note: once I edited the linear layer pruning to not prune the last layer, this problem was resolved.

However, pruning the linear layer caused more drastic performance decreases, so changed it to only prune the convolutional layers.

Note: pruning causes poor performance. Things to try: reset optimizer before pruning.

#### Age

In [6]:
task = 'age'
num_epochs = 10
with open('models/model_variants/model_score_lookup_singletask.tsv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['Task', 'prune_pct', 'mean_latency', 'std_latency', 'age_r2', 'gender_f1', 'ethnicity_f1'])
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 5.56717, train loss: 10.11769
Epoch 1, val loss: 5.56717 -> 5.34095, train loss: 6.54983
Epoch 2, val loss: 5.87016, train loss: 5.28468
Epoch 3, val loss: 5.34095 -> 4.65103, train loss: 4.44471
Epoch 4, val loss: 4.65103 -> 4.61679, train loss: 4.04575
Epoch 5, val loss: 4.61679 -> 4.17793, train loss: 3.34958
Epoch 6, val loss: 4.84131, train loss: 3.15393
Epoch 7, val loss: 4.78454, train loss: 2.74433
Epoch 8, val loss: 4.88639, train loss: 2.40511
Epoch 9, val loss: 4.37812, train loss: 2.30983
-------------- Fine-tuning Pruned Model -------------
Epoch 0, val loss: inf -> 8.02549, train loss: 3.23882
Epoch 1, val loss: 8.02549 -> 3.94230, train loss: 2.75208
Epoch 2, val loss: 4.08596, train loss: 2.39928
Epoch 3, val loss: 4.68539, train loss: 2.17427
Epoch 4, val loss: 3.94230 -> 3.75982, train loss: 1.89224
Epoch 5, val 

Epoch 1, val loss: 8.40090 -> 6.26513, train loss: 7.93040
Epoch 2, val loss: 6.67422, train loss: 6.29963
Epoch 3, val loss: 6.26513 -> 5.20368, train loss: 5.30288
Epoch 4, val loss: 5.43756, train loss: 4.52305
Epoch 5, val loss: 5.20368 -> 5.18966, train loss: 4.13952
Epoch 6, val loss: 5.18966 -> 5.09850, train loss: 3.58393
Epoch 7, val loss: 5.13607, train loss: 3.35669
Epoch 8, val loss: 5.54167, train loss: 2.99933
Epoch 9, val loss: 5.64975, train loss: 2.81623

--------------------------------- Prune 70 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 5.75263, train loss: 10.52901
Epoch 1, val loss: 6.34998, train loss: 6.42953
Epoch 2, val loss: 5.75263 -> 4.79656, train loss: 5.36921
Epoch 3, val loss: 4.79656 -> 4.43999, train loss: 4.71622
Epoch 4, val loss: 5.09162, train loss: 4.08518
Epoch 5, val loss: 6.18765, train loss: 3.50601
Epoch 6, val loss: 4.43999 -> 4.08426, train loss: 3.31681
Epoch 7, val 

#### Gender

In [7]:
task = 'gender'
num_epochs = 10

for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.01855, train loss: 0.02370
Epoch 1, val loss: 0.01855 -> 0.01581, train loss: 0.01750
Epoch 2, val loss: 0.02191, train loss: 0.01515
Epoch 3, val loss: 0.01592, train loss: 0.01489
Epoch 4, val loss: 0.01682, train loss: 0.01264
Epoch 5, val loss: 0.01581 -> 0.01332, train loss: 0.01152
Epoch 6, val loss: 0.01385, train loss: 0.00972
Epoch 7, val loss: 0.01349, train loss: 0.00916
Epoch 8, val loss: 0.01485, train loss: 0.00728
Epoch 9, val loss: 0.01685, train loss: 0.00780
-------------- Fine-tuning Pruned Model -------------
Epoch 0, val loss: inf -> 0.01373, train loss: 0.01066
Epoch 1, val loss: 0.01407, train loss: 0.00906
Epoch 2, val loss: 0.01486, train loss: 0.00984
Epoch 3, val loss: 0.01643, train loss: 0.00744
Epoch 4, val loss: 0.01750, train loss: 0.00504
Epoch 5, val loss: 0.01669, train loss: 0.00456
Epoch 6, v

Epoch 6, val loss: 0.02405, train loss: 0.00923
Epoch 7, val loss: 0.01823, train loss: 0.00799
Epoch 8, val loss: 0.01708, train loss: 0.00738
Epoch 9, val loss: 0.01750, train loss: 0.00664

--------------------------------- Prune 70 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.02250, train loss: 0.02383
Epoch 1, val loss: 0.02250 -> 0.01722, train loss: 0.01790
Epoch 2, val loss: 0.01722 -> 0.01380, train loss: 0.01564
Epoch 3, val loss: 0.01380 -> 0.01372, train loss: 0.01537
Epoch 4, val loss: 0.01387, train loss: 0.01275
Epoch 5, val loss: 0.01372 -> 0.01360, train loss: 0.01241
Epoch 6, val loss: 0.01377, train loss: 0.00981
Epoch 7, val loss: 0.01449, train loss: 0.00865
Epoch 8, val loss: 0.02866, train loss: 0.00850
Epoch 9, val loss: 0.01414, train loss: 0.01011
-------------- Fine-tuning Pruned Model -------------
Epoch 0, val loss: inf -> 0.02191, train loss: 0.02958
Epoch 1, val loss: 0.02191 -> 0.01

#### Ethnicity

In [8]:
task = 'ethnicity'
num_epochs = 10
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.04358, train loss: 0.06321
Epoch 1, val loss: 0.04358 -> 0.04009, train loss: 0.04801
Epoch 2, val loss: 0.04009 -> 0.03717, train loss: 0.04229
Epoch 3, val loss: 0.03729, train loss: 0.03783
Epoch 4, val loss: 0.03717 -> 0.03474, train loss: 0.03464
Epoch 5, val loss: 0.03596, train loss: 0.03170
Epoch 6, val loss: 0.03474 -> 0.03464, train loss: 0.02794
Epoch 7, val loss: 0.03677, train loss: 0.02418
Epoch 8, val loss: 0.03700, train loss: 0.02084
Epoch 9, val loss: 0.03949, train loss: 0.01792
-------------- Fine-tuning Pruned Model -------------
Epoch 0, val loss: inf -> 0.03643, train loss: 0.02425
Epoch 1, val loss: 0.03911, train loss: 0.02093
Epoch 2, val loss: 0.04328, train loss: 0.01845
Epoch 3, val loss: 0.03954, train loss: 0.01574
Epoch 4, val loss: 0.04666, train loss: 0.01407
Epoch 5, val loss: 0.04500, train lo

Epoch 4, val loss: 0.04448, train loss: 0.03494
Epoch 5, val loss: 0.04178, train loss: 0.03242
Epoch 6, val loss: 0.04094, train loss: 0.02951
Epoch 7, val loss: 0.03955 -> 0.03833, train loss: 0.02737
Epoch 8, val loss: 0.04147, train loss: 0.02544
Epoch 9, val loss: 0.04621, train loss: 0.02286

--------------------------------- Prune 70 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.04540, train loss: 0.06228
Epoch 1, val loss: 0.04540 -> 0.04092, train loss: 0.04689
Epoch 2, val loss: 0.04092 -> 0.03869, train loss: 0.04068
Epoch 3, val loss: 0.03869 -> 0.03676, train loss: 0.03773
Epoch 4, val loss: 0.03676 -> 0.03436, train loss: 0.03284
Epoch 5, val loss: 0.03474, train loss: 0.03039
Epoch 6, val loss: 0.03572, train loss: 0.02698
Epoch 7, val loss: 0.04387, train loss: 0.02360
Epoch 8, val loss: 0.04132, train loss: 0.02037
Epoch 9, val loss: 0.04138, train loss: 0.01718
-------------- Fine-tuning Pruned Mo